In [23]:
using Agents, Random, Distributions, UnicodePlots

Random.seed!(123);

In [24]:
mutable struct Human <: AbstractAgent
           id :: Int 
       status :: Symbol # :susceptible, :infected, or :removed
    infection :: Float64
end

agent_properties = [:status, :infection]

2-element Array{Symbol,1}:
 :status   
 :infection

In [25]:
function SIRModel(;
                              humans = 1000,         # Number of susceptible
                            infected = 10,           # Number of infected
                        contact_rate = 0.1,
                      infection_rate = 0.1,
                           time_step = 1.0           # hours
                  )   

    model_properties = Dict(
                            :contact_rate   => contact_rate,
                            :infection_rate => infection_rate,
                            :total          => humans,
                            :infected       => infected,
                            :p_infection    => 0.0,
                            :time_step      => time_step,
                           )   

    model = ABM(Human, scheduler=fastest, properties=model_properties)

    # Initialize susceptible population
    susceptible = humans - infected

    for i = 1:susceptible
        add_agent!(model, :susceptible, 0.0)
    end 

    # Initialize infected population
    for i = 1:infected
        add_agent!(model, :infected, 0)
    end 

    return model
end

model = SIRModel() # test whether function works...

AgentBasedModel with 1000 agents of type Human
 no space
 scheduler: fastest
 properties: Dict{Symbol,Real}(:infection_rate => 0.1,:p_infection => 0.0,:contact_rate => 0.1,:infected => 10,:total => 1000,:time_step => 1.0)

In [26]:
function agent_step!(agent, m)
    
    if agent.status == :removed
        return nothing
        
    elseif agent.status  == :susceptible
        
        # Define a stochastic process that determines whether the 
        # agent becomes infected.
        
        # The (random) number of infected people this agent contacts
        contacted = round(Int, rand(Gamma(m.infected, m.contact_rate * m.time_step)))
        
        # The probability that a susceptible is infected...
        p_infected = 1 - cdf(Binomial(contacted, m.infection_rate), 1)
        
        if rand(Bernoulli(p_infected))
            agent.status = :infected
        end
        
    elseif agent.status == :infected
        
        agent.infection += m.time_step
        
        # agent may become removed
    end 

    return nothing
end

agent_step! (generic function with 1 method)

In [ ]:
model = SIRModel(humans=100000, infected=10)
data = step!(model, agent_step!, 100, agent_properties)

infected = filter(x -> x.status === :infected, data)

In [ ]:
infection = infected[!, :infection]
UnicodePlots.histogram(infection)